# MNIST Notebook Anleitung

Wenn ihr es in Colab laufen lassen wollt, müsst ihr nachdem ihr das condition_INNs-Repository in einen Ordner eurer Wahl (z.B. CINN) clonen, die Ordner "images" mit Unterordnern "pca" and "samples" erstellen und darauf entsprechend in den Dateien "eval.py" und "config.py" verlinken. 
Wenn ihr das vortrainierte Modell aus dem untrigen Link verwenden wollt, müsst ihr auch darauf in "config.py" verlinken. Die Dateipfade gebt ihr entweder absolut in eurem Google Drive mit "/content/gdrive/MyDrive/CINN/conditional_INNs/mnist_cINN/..." and oder relativ zu dem Verzeichnis in dem ihr euch gerade im Juypter Notebook befindet. 

Checkpoint for the MNIST model:

https://drive.google.com/file/d/1Vf8RFX-n-HvBwgUTFPFcMBvf1kfBSEHg

(Simply download, and set load_file to the file location in config.py)
Colorization on 'Color-MNIST'

This folder also contains an additional experiment: 'color-MNIST' (set colorize = True in config.py)

The color-MNIST dataset contains color images, where each image is an MNIST digit, colorized in a certain color scheme corresponding to the digit, with some random fluctuations. The task is to generate color images, the condition is black and white. The dataset can be generated by cd color_mnist_data; python color_mnist.py.

The model uses a small conditioning network to extract semantic information (which digit) from the condition. The conditioning network can also be pretrained by running python -m cond_net.py.

# Colorization

Hier geht ihr ähnlich vor, die beiden vortrainierten Modelle (mit ohne ohne vortrainiertes Conditioning Network) findet ihr hier:



Model checkpoint used for the paper:

    https://drive.google.com/open?id=1gpHHtT7EcCoEqTzaUmDImp_tyB7vSKIN

Pretrained cond. net (only necessary if you want to train yourself, but not do the pretraining):

    https://drive.google.com/open?id=1YQkOf03kK7-ZNDGJmVFloF_hZvnmjc0_

Anschließend müsst ihr im ersten Schritt den bilateral filter mit Cython kreieren (d.h. Python zu C kompilieren):

  python setup.py build_ext --inplace


In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/My Drive/CINN

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/CINN


Die folgenden Zeile nur beim ersten Mal ausführen, um das conditional INN github in den Ordner MNIST zu clonen

In [ ]:
#%cd /content/gdrive/My Drive/MNIST
#!git clone https://github.com/VLL-HD/conditional_INNs

Hiermit die richtige Version von FrEIA installieren

In [2]:
pip install git+https://github.com/VLL-HD/FrEIA.git@v0.2

  Cloning https://github.com/VLL-HD/FrEIA.git (to revision v0.2) to /tmp/pip-req-build-1u4ys9a8
  Running command git clone -q https://github.com/VLL-HD/FrEIA.git /tmp/pip-req-build-1u4ys9a8
  Running command git checkout -q 033f6fd6eb4d2ddbc98f746a9ef5f8c619587abe


In [5]:
!python "conditional_INNs/mnist_cINN/eval.py"

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid poten

Die Option N bestimmt wie viele Epochen das Modell trainiert wird. 

In [ ]:
!python "conditional_INNs/mnist_cINN/train.py" -N 100

Config options:

  add_image_noise          	0.15
  batch_size               	512
  betas                    	(0.9, 0.999)
  checkpoint_on_error      	True
  checkpoint_save_interval 	360
  checkpoint_save_overwrite	True
  clamping                 	1.5
  colorize                 	False
  cond_net_file            	
  cond_width               	64
  data_mean                	0.0
  data_std                 	1.0
  decay_by                 	0.01
  do_fwd                   	True
  do_rev                   	False
  fc_dropout               	0.0
  filename                 	/content/gdrive/MyDrive/MNIST/conditional_INNs/mnist_cINN/output/mnist_cinn.pt
  img_dims                 	(28, 28)
  init_scale               	0.03
  internal_width           	512
  live_visualization       	False
  load_file                	/content/gdrive/MyDrive/MNIST/mnist_cinn.pt
  loss_names               	['L', 'L_rev']
  lr                       	0.0001
  n_blocks                 	24
  n_epochs                 	100
 

# Colorisation

Generiere hier den Filter. Ändere in setup.py die setup-Zeile zu 

```
setup(
    cmdclass = {'build_ext': build_ext},
    ext_modules = [Extension("joint_bilateral_filter",
                             sources=["conditional_INNs/colorization_cINN/joint_bilateral_filter.pyx", "conditional_INNs/colorization_cINN/joint_bilateral.c"],
                             include_dirs=[np.get_include()])],
)
```

Dann wird eine .so - Datei im Root-Verzeichnis (bei mir CINN) erstellt. Diese muss man dann wieder in den conditiona_INNs/colorization_cINN-Ordner kopieren. 


In [16]:
!python "conditional_INNs/colorization_cINN/setup.py" build_ext --inplace

running build_ext
cythoning conditional_INNs/colorization_cINN/joint_bilateral_filter.pyx to conditional_INNs/colorization_cINN/joint_bilateral_filter.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/gdrive/MyDrive/CINN/conditional_INNs/colorization_cINN/joint_bilateral_filter.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'joint_bilateral_filter' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/conditional_INNs
creating build/temp.linux-x86_64-3.7/conditional_INNs/colorization_cINN
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-

In [28]:
!python "conditional_INNs/colorization_cINN/eval.py"

Traceback (most recent call last):
  File "conditional_INNs/colorization_cINN/eval.py", line 35, in <module>
    import model
  File "/content/gdrive/My Drive/CINN/conditional_INNs/colorization_cINN/model.py", line 11, in <module>
    import data
  File "/content/gdrive/My Drive/CINN/conditional_INNs/colorization_cINN/data.py", line 101, in <module>
    with open('./imagenet/training_images.txt') as f:
FileNotFoundError: [Errno 2] No such file or directory: './imagenet/training_images.txt'


In [29]:
!python "conditional_INNs/colorization_minimal_example/eval.py"

Traceback (most recent call last):
  File "conditional_INNs/colorization_minimal_example/eval.py", line 10, in <module>
    import data
  File "/content/gdrive/MyDrive/CINN/conditional_INNs/colorization_minimal_example/data.py", line 68, in <module>
    test_all = torch.stack(list(test_data), 0).cuda()
  File "/content/gdrive/MyDrive/CINN/conditional_INNs/colorization_minimal_example/data.py", line 42, in __getitem__
    im = Image.open(self.files[idx])
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: './train_data_128/1.jpg'
